In [5]:
import pandas as pd
import numpy as np

In [6]:
import pandas as pd

# 1. Chargement du fichier
df = pd.read_csv("data/reviews83325.csv")

# 2. Conversion de la date en format datetime pour pouvoir trier
# Pandas est assez intelligent pour détecter le format, sinon on peut spécifier format='%Y-%m-%d'
df['date_review'] = pd.to_datetime(df['date_review'])

# 3. Tri des données : par destination, puis par date (de la plus récente à la plus ancienne)
df_sorted = df.sort_values(by=['idplace', 'date_review'], ascending=[True, False])

# 4. Groupement par idplace et sélection des 100 premières lignes pour chaque groupe
df_cleaned = df_sorted.groupby('idplace').head(100)

# 5. (Optionnel) Vérification
print(f"Nombre d'avis avant : {len(df)}")
print(f"Nombre d'avis après : {len(df_cleaned)}")
print(f"Nombre moyen d'avis par lieu : {df_cleaned.groupby('idplace').size().mean():.1f}")
print(f"Maximum d'avis pour un lieu : {df_cleaned.groupby('idplace').size().max()}")
print(f"Minimum d'avis pour un lieu : {df_cleaned.groupby('idplace').size().min()}")

# 6. Sauvegarde du nouveau fichier nettoyé
df_cleaned.to_csv('review_filtered_100.csv', index=False)

C:\Users\Thomas\AppData\Local\Temp\ipykernel_18404\3611611429.py:4: DtypeWarning: Columns (15,16,17,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/reviews83325.csv")
C:\Users\Thomas\AppData\Local\Temp\ipykernel_18404\3611611429.py:8: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['date_review'] = pd.to_datetime(df['date_review'])


Nombre d'avis avant : 340385
Nombre d'avis après : 72588
Nombre moyen d'avis par lieu : 33.6
Maximum d'avis pour un lieu : 100
Minimum d'avis pour un lieu : 1


### Merging datasets

In [10]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np

# 1. Charger les données nettoyées
df = pd.read_csv('review_filtered_100.csv')
df_places = pd.read_csv('data/Tripadvisor.csv')

# 2. Générer les vecteurs (Embeddings)
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
print("Encodage des avis en vecteurs...")


# On encode tous les avis du CSV
# corpus_embeddings = model.encode(df['review'].tolist(), show_progress_bar=True)

# # Sauvegarder les vecteurs
# np.save('avis_embeddings.npy', corpus_embeddings)

# # Sauvegarder l'index des avis pour garder la correspondance
# df.to_csv('df_final_indexed.csv', index=False)

corpus_embeddings = np.load('avis_embeddings.npy')
df = pd.read_csv('df_final_indexed.csv')

# 3. Fonction de recommandation
def recommander_destination(nouvel_avis, top_n=3):
    # Vectoriser l'avis d'entrée
    query_embedding = model.encode([nouvel_avis])
    
    # Calculer la similarité entre l'input et toute la base
    similarities = cosine_similarity(query_embedding, corpus_embeddings)[0]
    
    # Récupérer les indices des avis les plus proches
    top_indices = np.argsort(similarities)[-20:][::-1] # on prend les 20 meilleurs avis
    
    # Créer un score par lieu (idplace)
    scores_lieux = {}
    for idx in top_indices:
        id_lieu = df.iloc[idx]['idplace']
        score = similarities[idx]
        scores_lieux[id_lieu] = scores_lieux.get(id_lieu, 0) + score
        
    # Trier les lieux par score cumulé
    lieux_tries = sorted(scores_lieux.items(), key=lambda x: x[1], reverse=True)
    
    # Afficher les noms des lieux
    for id_l, score in lieux_tries[:top_n]:
        nom_lieu = df_places[df_places['id'] == id_l]['nom'].values[0]
        print(f"Destination recommandée : {nom_lieu} (Score: {score:.4f})")

# Test
recommander_destination("Personally I think it is the most beautiful square of Paris. Well maintained and the area around it gives you opportunities to grab a bite to eat as well.")

Loading weights: 100%|██████████| 199/199 [00:00<00:00, 775.34it/s, Materializing param=pooler.dense.weight]                               
BertModel LOAD REPORT from: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Encodage des avis en vecteurs...
Destination recommandée : Place des Vosges (Score: 4.9436)
Destination recommandée : 4ème arrondissement (Score: 3.1766)
Destination recommandée : Square de l'Ile de France (Score: 1.5784)
